Laboratorio 1 IA 2
Nombres:José Fernando
Apellidos:Alfaro Ayzama

1.-Aplicar Pytorch para construir un MLP que permita clasificar almenos 5 clases(imagenes,cualquier cosa).

2.-El mlp debe contener almenos unas 50 unidades neuronales en su capa oculta.

3.-se debe entrenar con almenos 1000 epochs()esplicar que pasa

4.-Guardar checkpoints de entrenamiento cada 20 epochs

5.-utilizar los distintos checkpoints para reflexionar sobre los resultados(saber los resultados).

6.-Aplicar las formas de exportacion de modelos.

7.-Caracteristicas del datased(entradas n.X>=10 m>=10000 Y<=5)

8.-Se debe coordinar entre estudiantes para evitar que dos o mas estudiantes utilizen el mismo dataset.

9.-Utilizar de manera obligatoria objetos dataset y dataloader en la implementacion.

Dataset Utilizado en este cuadernillo:
https://www.kaggle.com/datasets/nimapourmoradi/dry-bean-dataset-classification

link de github: repocitorio:https://github.com/Josefer98/Inteligencia_Artifical_2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os

# Cargar el conjunto de datos
data = pd.read_csv("Dry_Bean_Dataset.csv")

# Preprocesamiento de datos
label_encoder = LabelEncoder()
data['Class'] = label_encoder.fit_transform(data['Class'])

X = data.drop('Class', axis=1).values
y = data['Class'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

class BeanDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = BeanDataset(X_train, y_train)
test_dataset = BeanDataset(X_test, y_test)

# Definir el modelo MLP
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

input_size = X_train.shape[1]
hidden_size = 50
num_classes = len(label_encoder.classes_)

model = MLP(input_size, hidden_size, num_classes)

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
num_epochs = 1000
checkpoint_interval = 20
checkpoint_dir = 'checkpoints'

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in DataLoader(train_dataset, batch_size=32, shuffle=True):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

    # Guardar puntos de control
    if (epoch + 1) % checkpoint_interval == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch+1}.pt')
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': epoch_loss
        }, checkpoint_path)

# Evaluación del modelo
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in DataLoader(test_dataset, batch_size=32):
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Accuracy on test set: {accuracy:.4f}')


Epoch [1/1000], Loss: 276.0800
Epoch [2/1000], Loss: 76.0071
Epoch [3/1000], Loss: 73.2413
Epoch [4/1000], Loss: 46.1005
Epoch [5/1000], Loss: 50.5538
Epoch [6/1000], Loss: 40.5256
Epoch [7/1000], Loss: 30.3363
Epoch [8/1000], Loss: 34.4130
Epoch [9/1000], Loss: 42.2705
Epoch [10/1000], Loss: 24.7135
Epoch [11/1000], Loss: 23.8361
Epoch [12/1000], Loss: 28.9978
Epoch [13/1000], Loss: 23.9330
Epoch [14/1000], Loss: 24.3575
Epoch [15/1000], Loss: 21.2759
Epoch [16/1000], Loss: 22.2859
Epoch [17/1000], Loss: 19.3822
Epoch [18/1000], Loss: 23.7937
Epoch [19/1000], Loss: 17.8766
Epoch [20/1000], Loss: 25.4990
Epoch [21/1000], Loss: 14.0575
Epoch [22/1000], Loss: 16.7596
Epoch [23/1000], Loss: 17.5811
Epoch [24/1000], Loss: 16.8116
Epoch [25/1000], Loss: 12.4236
Epoch [26/1000], Loss: 16.4822
Epoch [27/1000], Loss: 15.7990
Epoch [28/1000], Loss: 14.1537
Epoch [29/1000], Loss: 14.6261
Epoch [30/1000], Loss: 15.0612
Epoch [31/1000], Loss: 15.9890
Epoch [32/1000], Loss: 12.3926
Epoch [33/1000],

GUardar Modelo

In [ ]:
torch.save(model, 'model.pth')


Cargar el modelo

In [ ]:
model = torch.load('model.pth')


Guarda los pesos del modelo

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')


Exportar en formato ONNX

In [ ]:
pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 52.9 MB/s eta 0:00:00


In [ ]:
input_sample = torch.randn(1, input_size)  # Crear un tensor de muestra de entrada
torch.onnx.export(model, input_sample, 'model.onnx')


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os

# Cargar el conjunto de datos
data = pd.read_csv("Dry_Bean_Dataset.csv")

# Preprocesamiento de datos
label_encoder = LabelEncoder()
data['Class'] = label_encoder.fit_transform(data['Class'])

X = data.drop('Class', axis=1).values
y = data['Class'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

class DatasetPersonalizado(torch.utils.data.Dataset):
    # constructor
    def __init__(self, X, Y):
        self.X = torch.from_numpy(X).float().cuda()
        self.Y = torch.from_numpy(Y).long().cuda()
    # devolvemos el número de datos en el dataset
    def __len__(self):
        return len(self.X)
    # devolvemos el elemento `ix` del dataset
    def __getitem__(self, ix):
        return self.X[ix], self.Y[ix]

train_dataset = DatasetPersonalizado(X_train, y_train)
test_dataset = DatasetPersonalizado(X_test, y_test)

# Definir el modelo MLP
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

input_size = X_train.shape[1]
hidden_size = 50
num_classes = len(label_encoder.classes_)

model = MLP(input_size, hidden_size, num_classes).cuda()

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
num_epochs = 1000
checkpoint_interval = 20
checkpoint_dir = 'checkpoints'

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in torch.utils.data.DataLoader(train_dataset, batch_size=62, shuffle=True):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

    # Guardar puntos de control
    if (epoch + 1) % checkpoint_interval == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch+1}.pt')
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': epoch_loss
        }, checkpoint_path)

# Evaluación del modelo
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in torch.utils.data.DataLoader(test_dataset, batch_size=62):
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Accuracy on test set: {accuracy:.4f}')


Epoch [1/1000], Loss: 650.2891
Epoch [2/1000], Loss: 64.9436
Epoch [3/1000], Loss: 60.4412
Epoch [4/1000], Loss: 45.5871
Epoch [5/1000], Loss: 46.0133
Epoch [6/1000], Loss: 41.6313
Epoch [7/1000], Loss: 42.9285
Epoch [8/1000], Loss: 42.4867
Epoch [9/1000], Loss: 32.8571
Epoch [10/1000], Loss: 36.4204
Epoch [11/1000], Loss: 32.1708
Epoch [12/1000], Loss: 28.3282
Epoch [13/1000], Loss: 20.3722
Epoch [14/1000], Loss: 22.3866
Epoch [15/1000], Loss: 21.9209
Epoch [16/1000], Loss: 28.4712
Epoch [17/1000], Loss: 22.7133
Epoch [18/1000], Loss: 17.4238
Epoch [19/1000], Loss: 25.1456
Epoch [20/1000], Loss: 19.9404
Epoch [21/1000], Loss: 14.5912
Epoch [22/1000], Loss: 10.8420
Epoch [23/1000], Loss: 22.6464
Epoch [24/1000], Loss: 16.6444
Epoch [25/1000], Loss: 12.6728
Epoch [26/1000], Loss: 11.6991
Epoch [27/1000], Loss: 22.5936
Epoch [28/1000], Loss: 17.9194
Epoch [29/1000], Loss: 18.5044
Epoch [30/1000], Loss: 16.7435
Epoch [31/1000], Loss: 16.5026
Epoch [32/1000], Loss: 15.5127
Epoch [33/1000],